# Example _de novo_ RADseq assembly using _pyRAD_

# Modification to start looking at Ostrea data

----------  

Please direct questions about _pyRAD_ analyses to the google group thread ([link](https://groups.google.com/forum/#!forum/pyrad-users))  

--------------  



+  This tutorial is meant as a walkthrough for a single-end RADseq analyses. If you have not yet read the [__full tutorial__](http://www.dereneaton.com/software/pyrad), you should start there for a broader description of how _pyRAD_ works. If you are new to RADseq analyses, this tutorial will provide a simple overview of how to execute _pyRAD_, what the data files look like, and how to check that your analysis is working, and the expected output formats.  



+  Each cell in this tutorial begins with the header (%%bash) indicating that the code should be executed in a command line shell, for example by copying and pasting the text into your terminal (but excluding the %%bash header).

-------------  


Begin by executing the command below. This will download an example simulated RADseq data set and unarchive it into your current directory.

In [2]:
pwd

u'/Users/sr320/git-repos/nb-2016/util'

In [1]:
cd /Volumes/web-1/halfshell/working-directory


/Volumes/web-1/halfshell/working-directory


In [2]:
cd 16-04-15b

/Volumes/web-1/halfshell/working-directory/16-04-15b


In [3]:
ls | head

1HL_1A_1.fq.gz*
1HL_1A_2.fq.gz*
1HL_2A_1.fq.gz*
1HL_2A_2.fq.gz*
1HL_3A_1.fq.gz*
1HL_3A_2.fq.gz*
1HL_4A_1.fq.gz*
1HL_4A_2.fq.gz*
1NF_1A_1.fq.gz*
1NF_1A_2.fq.gz*


#### We begin by creating the params.txt file which is used to set all parameters for an analysis.

------------   

The params file lists on each line one parameter followed by a __##__ mark, after which any comments can be left. In the comments section there is a description of the parameter and in parentheses the step of the analysis affected by the parameter. Lines 1-12 are required, the remaining lines are optional. The params.txt file is further described in the general tutorial.

#### Let's take a look at the default settings. 

In [4]:
%%bash
cat params.txt

==** parameter inputs for pyRAD version 3.0.64  **======================== affected step ==
./                        ## 1. Working directory                                 (all)
./*.fastq.gz              ## 2. Loc. of non-demultiplexed files (if not line 18)  (s1)
./*.barcodes              ## 3. Loc. of barcode file (if not line 18)             (s1)
/Applications/bioinfo/vsearch-1.11.1-osx-x86_64/bin/vsearch                   ## 4. command (or path) to call vsearch (or usearch)    (s3,s6)
/Applications/bioinfo/muscle3.8.31_i86darwin64                    ## 5. command (or path) to call muscle                  (s3,s7)
CWGC                     ## 6. Restriction overhang (e.g., C|TGCAG -> TGCAG)     (s1,s2)
6                         ## 7. N processors (parallel)                           (all)
5                         ## 8. Mindepth: min coverage for a cluster              (s4,s5)
4                         ## 9. NQual: max # sites with qual < 20 (or see line 20)(s2)
.88                 

#### To change parameters you can edit params.txt in any text editor. Here to automate things I use the script below.

#### Let's have a look at the changes:

--------------   

__Let's take a look at what the raw data look like.__

Your input data will be in fastQ format, usually ending in .fq or .fastq. Your data could be split among multiple files, or all within a single file (de-multiplexing goes much faster if they happen to be split into multiple files). The file/s may be compressed with gzip so that they have a .gz ending, but they do not need to be. The location of these files should be entered on line 2 of the params file. Below are the first three reads in the example file.

In [6]:
ls

1HL_1A_1.fq.gz* 1HL_3A_2.fq.gz* 1NF_2A_1.fq.gz* 1NF_5A_2.fq.gz* 1SN_3A_1.fq.gz*
1HL_1A_2.fq.gz* 1HL_4A_1.fq.gz* 1NF_2A_2.fq.gz* 1SN_1A_1.fq.gz* 1SN_3A_2.fq.gz*
1HL_2A_1.fq.gz* 1HL_4A_2.fq.gz* 1NF_4A_1.fq.gz* 1SN_1A_2.fq.gz* 1SN_4A_1.fq.gz*
1HL_2A_2.fq.gz* 1NF_1A_1.fq.gz* 1NF_4A_2.fq.gz* 1SN_2A_1.fq.gz* 1SN_4A_2.fq.gz*
1HL_3A_1.fq.gz* 1NF_1A_2.fq.gz* 1NF_5A_1.fq.gz* 1SN_2A_2.fq.gz* params.txt*


In [16]:
!gunzip *.gz

^C


In [17]:
ls | head

1HL_10A_1.fq*
1HL_10A_2.fq*
1HL_10A_2.fq.gz*
1HL_11A_1.fq.gz*
1HL_11A_2.fq.gz*
1HL_12A_1.fq.gz*
1HL_12A_2.fq.gz*
1HL_13A_1.fq.gz*
1HL_13A_2.fq.gz*
1HL_14A_1.fq.gz*


In [28]:
%%bash
less simRADs_R1.fastq | head -n 12 | cut -c 1-90

@lane1_fakedata0_R1_0 1:N:0:
TTTTAATGCAGTGAGTGGCCATGCAATATATATTTACGGGCGCATAGAGACCCTCAAGACTGCCAACCGGGTGAATCACTATTTGCTTAG
+
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
@lane1_fakedata0_R1_1 1:N:0:
TTTTAATGCAGTGAGTGGCCATGCAATATATATTTACGGGCGCATAGAGACCCTCAAGACTGCCAACCGGGTGAATCACTATTTGCTTAG
+
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
@lane1_fakedata0_R1_2 1:N:0:
TTTTAATGCAGTGAGTGGCCATGCAATATATATTTACGGGCGCATAGAGACCCTCAAGACTGCCAACCGGGTGAATCACTATTTGCTTAG
+
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB


------------   

Each read takes four lines. The first is the name of the read (its location on the plate). The second line contains the sequence data. The third line is a spacer. And the fourth line the quality scores for the base calls. In this case arbitrarily high since the data were simulated. 

These are 100 bp single-end reads prepared as RADseq. The first six bases form the barcode and the next five bases (TGCAG) the restriction site overhang. All following bases make up the sequence data. 

----------------   

## Step 1: de-multiplexing ##

In [ ]:
already done by bgi

### Step 2: quality filtering

In [9]:
%%bash
pyRAD -p params.txt -s 2



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------

	step 2: editing raw reads 
	........................

In [10]:
%%bash
ls edits/

1HL_1A_1.edit
1HL_1A_2.edit
1HL_2A_1.edit
1HL_2A_2.edit
1HL_3A_1.edit
1HL_3A_2.edit
1HL_4A_1.edit
1HL_4A_2.edit
1NF_1A_1.edit
1NF_1A_2.edit
1NF_2A_1.edit
1NF_2A_2.edit
1NF_4A_1.edit
1NF_4A_2.edit
1NF_5A_1.edit
1NF_5A_2.edit
1SN_1A_1.edit
1SN_1A_2.edit
1SN_2A_1.edit
1SN_2A_2.edit
1SN_3A_1.edit
1SN_3A_2.edit
1SN_4A_1.edit
1SN_4A_2.edit


The filtered data are written in fasta format (quality scores removed) into a new directory called edits/. Below I show a preview of the file which you can view most easily using the `less` command (I use `head` here to make it fit in the text window better).

In [13]:
%%bash
head -n 10 edits/1HL_1A_1.edit | cut -c 1-80

>1HL_1A_1_0_r1
CTGCAGTTATGGGTTTAGTAATTAGATTAGTTGTGGCTTGCTTAGAATATAATATTCTATTTCAAGCGCAACATATTGCA
>1HL_1A_1_1_r1
CTGCTTCACATGAGGAAAAAGTTCCAAAAGAAGACAACAGAAAAATGCCAATGAAGAGAATAAGAAATATCAATCAAAAA
>1HL_1A_1_2_r1
CTGCGGTGCCTGAGAAATAAAAAAAAGTTTGTTCTGCCTAAAGCTGAGATTGGTACAGTGGAACAAAGTGCTGAAGAAAG
>1HL_1A_1_3_r1
CTGCATTAAATCGGCATATTTTTATTTTTTATAGGATTGAATATATAAATTTTCAGAAATGAATACAAAAACCGAATAGC
>1HL_1A_1_4_r1
CTGCACATCAAGAAAGGGGCAGGGCCAACATCGGTAAGGAAGGGCAAGGTTCAGAAATAGGCAGGGCAAACAGTATGTCG


### Step 3: clustering within-samples

Step 3 de-replicates and then clusters reads within each sample by the set clustering threshold and writes the clusters to new files in a directory called clust.xx

In [14]:
%%bash
pyRAD -p params.txt -s 3



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	de-replicating files for clustering...

	step 3: within-sample clustering of 24 samples at 
	        '.88' similarity. Running 6 parallel jobs
	 	with up to 6 threads per job. If needed, 
		adjust to avoid CPU and MEM limits

	sample 1NF_2A_1 finished, 211397 loci
	sample 1SN_4A_1 finished, 223325 loci
	sample 1NF_4A_1 finished, 227834 loci
	sample 1SN_4A_2 finished, 262292 loci
	sample 1NF_2A_2 finished, 254170 loci
	sample 1NF_4A_2 finished, 273403 loci
	sample 1SN_2A_1 finished, 211909 loci
	sample 1SN_1A_2 finished, 233509 loci
	sample 1SN_1A_1 finished, 196225 loci
	sample 1SN_2A_2 finished, 250692 loci
	sample 1NF_5A_1 finished, 201878 loci
	sample 1HL_2A_2 finished, 220793 loci
	sample 1NF_5A_2 finished, 235210 loci
	sample 1HL_2A_1 finished, 188223 loci
	sample 1HL_4A_1 finished, 

Once again, I recommend you use the unix command 'less' to look at the clustS files. These contain each cluster separated by "//". For the first few clusters below you can see that there is one or two alleles in the cluster and one or a few reads that contained a (simulated) sequencing error. 

In [44]:
%%bash
less clust.85/1A0.clustS.gz | head -n 26 | cut -c 1-80

"clust.85/1A0.clustS.gz" may be a binary file.  See it anyway? 

---------------


The stats output tells you how many clusters were found, and their mean depth of coverage. It also tells you how many pass  your minimum depth setting. You can use this information to decide if you wish to increase or decrease the mindepth before it is applied for making consensus base calls in steps 4 & 5.

In [15]:
%%bash
head -n 40 stats/s3.clusters.txt


taxa	total	dpt.me	dpt.sd	d>4.tot	d>4.me	d>4.sd	badpairs
1HL_1A_1	180183	9.1	57.224	65738	21.779	93.383	0
1HL_1A_2	207572	7.509	38.931	64027	20.606	68.289	0
1HL_2A_1	188223	10.441	62.134	72867	24.079	98.323	0
1HL_2A_2	220793	8.49	41.232	71187	22.868	70.469	0
1HL_3A_1	214916	7.441	38.217	66187	20.338	67.083	0
1HL_3A_2	241082	6.418	31.632	64417	19.647	59.191	0
1HL_4A_1	187837	9.827	52.59	71766	22.797	83.456	0
1HL_4A_2	217303	8.232	45.561	69785	22.148	78.592	0
1NF_1A_1	127769	7.742	35.842	45123	18.547	58.781	0
1NF_1A_2	145020	6.633	29.957	43746	18.055	52.782	0
1NF_2A_1	211397	11.667	65.516	85433	26.21	101.314	0
1NF_2A_2	254170	9.375	51.79	83372	25.271	88.313	0
1NF_4A_1	227834	11.261	66.186	91019	25.482	103.087	0
1NF_4A_2	273403	9.101	54.68	88937	24.634	93.979	0
1NF_5A_1	201878	9.78	48.097	77386	22.607	75.937	0
1NF_5A_2	235210	8.207	41.016	75595	22.064	70.352	0
1SN_1A_1	196225	11.115	62.818	77527	25.362	98.238	0
1SN_1A_2	233509	9.044	51.384	75550	24.548	88.337	0
1SN_2A_1	211909	10.364	62.5

### Steps 4 & 5: Call consensus sequences

#### Step 4 jointly infers the error-rate and heterozygosity across samples.

In [16]:
%%bash
pyRAD -p params.txt -s 4



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	step 4: estimating error rate and heterozygosity
	........................

In [17]:
%%bash
less stats/Pi_E_estimate.txt

taxa	H	E
1NF_1A_1	0.01178287	0.00227395	
1NF_1A_2	0.01111578	0.00456467	
1HL_1A_1	0.01165588	0.00222957	
1HL_4A_1	0.01128766	0.00215498	
1HL_2A_1	0.01129892	0.00215894	
1SN_1A_1	0.01124951	0.00206785	
1HL_3A_1	0.01176524	0.00222189	
1NF_5A_1	0.0110808	0.00199742	
1SN_2A_1	0.01125617	0.00216491	
1NF_2A_1	0.0105777	0.00200486	
1SN_3A_1	0.01152874	0.00218917	
1SN_4A_1	0.01122477	0.00222883	
1NF_4A_1	0.01085647	0.00206061	
1HL_1A_2	0.01101551	0.00432483	
1HL_4A_2	0.01101026	0.00427231	
1HL_2A_2	0.01075435	0.00430961	
1HL_3A_2	0.01118699	0.00438399	
1NF_5A_2	0.01077547	0.00432624	
1SN_1A_2	0.01095606	0.00433219	
1SN_3A_2	0.01090279	0.00450492	
1SN_2A_2	0.01092085	0.00425974	
1NF_2A_2	0.01039075	0.00440895	
1SN_4A_2	0.01096988	0.00426638	
1NF_4A_2	0.0107695	0.00437047	


#### Step 5 calls consensus sequences using the parameters inferred above, and filters for paralogs.

In [18]:
%%bash
pyRAD -p params.txt -s 5



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	step 5: creating consensus seqs for 24 samples, using H=0.01110 E=0.00325
	........................

#### The stats output for step 5

In [19]:
%%bash
less stats/s5.consens.txt

taxon          	nloci	f1loci	f2loci	nsites	npoly	poly
1SN_3A_2	266870	68346	59729	5783777	26715	0.004619
1SN_1A_2	233509	75396	66194	6446729	30395	0.0047148
1SN_2A_2	250692	77262	67803	6595427	30717	0.0046573
1SN_4A_2	262292	83485	73488	7171633	33247	0.0046359
1NF_2A_2	254170	83191	73261	7149211	31778	0.004445
1NF_4A_2	273403	88764	78173	7633358	35859	0.0046977
1HL_3A_2	241082	64329	56407	5478855	26280	0.0047966
1NF_5A_2	235210	75474	66358	6463432	29405	0.0045494
1HL_1A_2	207572	63938	56129	5449230	26035	0.0047777
1HL_2A_2	220793	71082	62536	6092599	28086	0.0046099
1HL_4A_2	217303	69673	61284	5964488	27571	0.0046225
1NF_4A_1	227834	90819	83725	7838087	36674	0.0046789
1SN_3A_1	235474	70093	64185	5834728	27607	0.0047315
1SN_4A_1	223325	85604	78737	7206044	33832	0.0046949
1SN_2A_1	211909	79071	72755	6641453	31759	0.0047819
1NF_5A_1	201878	77245	70997	6628802	30404	0.0045867
1NF_2A_1	211397	85225	78715	7363184	32784	0.0044524
1HL_3A_1	214916	66084	60633	5460263	27167	0.0049754
1NF_1A_2	145

### Step 6: Cluster across samples

Step 6 clusters consensus sequences across samples. It will print its progress to the screen. This uses 6 threads by default. If you enter 0 for param 37 it will use all available processors. 

In [20]:
%%bash
pyRAD -p params.txt -s 6 

vsearch v1.11.1_osx_x86_64, 32.0GB RAM, 8 cores
https://github.com/torognes/vsearch


	finished clustering




     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	step 6: clustering across 24 samples at '.88' similarity 

Reading file /Volumes/web-1/halfshell/working-directory/16-04-15b/clust.88/cat.haplos_ 100%
155013399 nt in 1574785 seqs, min 33, max 153, avg 98
Counting unique k-mers 100%
Clustering 100%
Sorting clusters 100%
Writing clusters 100%
Clusters: 217149 Size min 1, max 62, avg 7.3
Singletons: 53358, 3.4% of seqs, 24.6% of clusters


## Step 7: Assemble final data sets

The final step is to output data only for the loci that you want to have included in your data set. This filters once again for potential paralogs or highly repetitive regions, and includes options to minimize the amount of missing data in the output. 

In [27]:
%%bash
pyRAD -p params.txt -s 7

	ingroup 1HL_1A_1,1HL_1A_2,1HL_2A_1,1HL_2A_2,1HL_3A_1,1HL_3A_2,1HL_4A_1,1HL_4A_2,1NF_1A_1,1NF_1A_2,1NF_2A_1,1NF_2A_2,1NF_4A_1,1NF_4A_2,1NF_5A_1,1NF_5A_2,1SN_1A_1,1SN_1A_2,1SN_2A_1,1SN_2A_2,1SN_3A_1,1SN_3A_2,1SN_4A_1,1SN_4A_2
	addon 
	exclude 
	
	final stats written to:
	 /Volumes/web-1/halfshell/working-directory/16-04-15b/stats/c88d6m4p3_2.stats
	output files being written to:
	 /Volumes/web-1/halfshell/working-directory/16-04-15b/outfiles/ directory





     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------

......

### Final stats output

In [29]:
%%bash
less stats/c88d6m4p3_2.stats



56612       ## loci with > minsp containing data
56426       ## loci with > minsp containing data & paralogs removed
56426       ## loci with > minsp containing data & paralogs removed & final filtering

## number of loci recovered in final data set for each taxon.
taxon	nloci
1HL_1A_1	39071
1HL_1A_2	37321
1HL_2A_1	41795
1HL_2A_2	40563
1HL_3A_1	39052
1HL_3A_2	37551
1HL_4A_1	41632
1HL_4A_2	40133
1NF_1A_1	30287
1NF_1A_2	28375
1NF_2A_1	45720
1NF_2A_2	44426
1NF_4A_1	46288
1NF_4A_2	45088
1NF_5A_1	43569
1NF_5A_2	42111
1SN_1A_1	43773
1SN_1A_2	42236
1SN_2A_1	43722
1SN_2A_2	42228
1SN_3A_1	39948
1SN_3A_2	38299
1SN_4A_1	45095
1SN_4A_2	43797


## nloci = number of loci with data for exactly ntaxa
## ntotal = number of loci for which at least ntaxa have data
ntaxa	nloci	saved	ntotal
1	-
2	-		-
3	-		-
4	-		-
5	-		-
6	-		-
7	-		-
8	-		-
9	-		-
10	5248	*	56426
11	3715	*	51178
12	4321	*	47463
13	3120	*	43142
14	3610	*	40022
15	2658	*	36412
16	3267	*	33754
17	2490	*	30487
18	3068	*	27997
19	2332	*	249

---------------  

## Output formats ##

We created 11 output files from our analysis. The standard two (.loci and .excluded_loci), as well as the 9 additional ones listed in the params file. These are all shown below.

In [30]:
%%bash 
ls outfiles/

c88d6m4p3.excluded_loci
c88d6m4p3.loci
c88d6m4p3_2.excluded_loci
c88d6m4p3_2.loci


### Loci format  
The ".loci" file contains each locus listed in a fasta-like format that also shows which sites are variable below each locus. Autapomorphies are listed as '-' and shared SNPs as '*'. This is a custom format that is human readable and also used as input to perform D-statistic tests in pyRAD. This is the easiest way to visualize your results. I recommend viewing the file with the command `less`. Below I use a head and cut to make it easy to view in this window.

In [ ]:
%%bash 
head -n 39 outfiles/c85m4p3.loci | cut -c 1-75

### PHY format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.phy | cut -c 1-85

### NEX format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.nex | cut -c 1-85

### Alleles format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.alleles| cut -c 1-85

### STRUCTURE (.str) format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.str | cut -c 1-20

### GENO (.geno) format (used in _Admixture_)

In [ ]:
%%bash 
head -n 40 outfiles/c85m4p3.geno 

### SNPs format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.snps | cut -c 1-85

### UNLINKED_SNPs format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.unlinked_snps | cut -c 1-85

## OTHER FORMATS  

You may also produce some more complicated formatting options that involve pooling individuals into groups or populations. This can be done for the "treemix" and "migrate" outputs, which are formatted for input into the programs _TreeMix_ and _migrate-n_, respectively. Grouping individuals into populations is done with the final lines of the params file as shown below, and similar to the assignment of individuals into clades for hierarchical clustering (see full tutorial). 

Each line designates a group, and has three arguments that are separated by space or tab. The first is the group name, the second is the minimum number of individuals that must have data in that group for a locus to be included in the output, and the third is a list of the members of that group. Lists of taxa can include comma-separated names and wildcard selectors, like below. Example:


In [ ]:
%%bash 
## append group designations to the params file
echo "pop1 4 1A0,1B0,1C0,1D0 " >> params.txt
echo "pop2 4 2E0,2F0,2G0,2H0 " >> params.txt
echo "pop3 4 3* " >> params.txt

## view params file
cat params.txt

## Creating population output files  
Now if we run _pyRAD_ with the 'm' (migrate) or 't' (treemix) output options, it will create their output files. 

In [ ]:
%%bash 
pyRAD -p params.txt -s 7

## TREEMIX format

In [ ]:
%%bash 
less outfiles/c85m4p3.treemix.gz | head -n 30

## MIGRATE-n FORMAT

In [ ]:
%%bash 
head -n 40 outfiles/c85m4p3.migrate | cut -c 1-85